<a href="https://colab.research.google.com/github/JoshStrother/DataVisualization/blob/main/Lesson6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


The first thing we need to do is grab the data from bike share stations (only the station ID and property type) and the bike share trips (station ID and duration of trip).

In [ ]:
%%bigquery --project data-viz-338716

SELECT station_id, property_type
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
LIMIT 50

,station_id,property_type
0,2576,None
1,2712,None
2,3381,None
3,3464,None
4,1001,None
5,1002,None
6,1003,None
7,1004,None
8,1005,None
9,1006,None


In [ ]:
%%bigquery --project data-viz-338716
SELECT start_station_id, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 10

,start_station_id,duration_minutes
0,1006,39
1,2712,31
2,2712,31
3,2712,30
4,2541,19
5,2541,17
6,2536,6
7,2712,0
8,2536,12
9,3464,47


Now im going to join the tables by keeping the property type and using the station ID as a relation and adding the duration of the trip based on where it starts from a new table.

In [ ]:
%%bigquery --project data-viz-338716
SELECT proptype.station_id, proptype.property_type, duration.duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` as proptype LEFT OUTER JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as duration
    ON proptype.station_id = duration.start_station_id

,station_id,property_type,duration_minutes
0,2576,None,88.0
1,2576,None,87.0
2,2576,None,87.0
3,2576,None,86.0
4,2576,None,56.0
...,...,...,...
1401010,3377,undetermined_parking,43.0
1401011,3377,undetermined_parking,43.0
1401012,3377,undetermined_parking,43.0
1401013,3377,undetermined_parking,43.0


Okay so I didnt try to slim it down to the data that we needed quite yet because I wanted to make sure it atleast worked. Now I will trim it down to the parkland and sidewalk.

In [ ]:
%%bigquery --project data-viz-338716
SELECT proptype.station_id, proptype.property_type, duration.duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` as proptype LEFT OUTER JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as duration
    ON proptype.station_id = duration.start_station_id
WHERE proptype.property_type = 'sidewalk' OR proptype.property_type = 'parkland'

,station_id,property_type,duration_minutes
0,3687,parkland,53.0
1,3687,parkland,68.0
2,3687,parkland,67.0
3,3687,parkland,66.0
4,3687,parkland,57.0
...,...,...,...
668368,2566,sidewalk,52.0
668369,2566,sidewalk,52.0
668370,2566,sidewalk,52.0
668371,2566,sidewalk,52.0


Now im going to get just the parkland and then just the sidewalk then calculate the AVG...I tried to do them at the same time but it was not working for me.

In [14]:
%%bigquery --project data-viz-338716
SELECT proptype.station_id as sidewalk_id, proptype.property_type as sidewalk_type, duration.duration_minutes as sidewalk_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` as proptype LEFT OUTER JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as duration
    ON proptype.station_id = duration.start_station_id
WHERE proptype.property_type = 'sidewalk'

,sidewalk_id,sidewalk_type,sidewalk_time
0,1111,sidewalk,NaN
1,11,sidewalk,NaN
2,4699,sidewalk,76.0
3,4699,sidewalk,75.0
4,4699,sidewalk,72.0
...,...,...,...
506310,2499,sidewalk,39.0
506311,2499,sidewalk,39.0
506312,2499,sidewalk,39.0
506313,2499,sidewalk,39.0


In [13]:
%%bigquery --project data-viz-338716
SELECT proptype.station_id as parkland_id, proptype.property_type as parkland_type, duration.duration_minutes as parkland_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` as proptype LEFT OUTER JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as duration
    ON proptype.station_id = duration.start_station_id
WHERE proptype.property_type = 'parkland'

,parkland_id,parkland_type,parkland_time
0,3687,parkland,68
1,3687,parkland,67
2,3687,parkland,66
3,3687,parkland,64
4,3687,parkland,63
...,...,...,...
162053,2707,parkland,60
162054,2707,parkland,60
162055,2707,parkland,60
162056,2707,parkland,60


Okay so I have them separated, now i can do a subquery and get the info I need.

In [22]:
%%bigquery --project data-viz-338716
SELECT SUM(sidewalk_time)/COUNT(sidewalk_type) as sidewalk_avg_min
FROM(SELECT proptype.station_id as sidewalk_id, proptype.property_type as sidewalk_type, duration.duration_minutes as sidewalk_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` as proptype LEFT OUTER JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as duration
    ON proptype.station_id = duration.start_station_id
WHERE proptype.property_type = 'sidewalk')

,sidewalk_avg_min
0,31.453196


In [23]:
%%bigquery --project data-viz-338716
SELECT SUM(parkland_time)/COUNT(parkland_type) as parkland_avg_min
FROM(SELECT proptype.station_id as parkland_id, proptype.property_type as parkland_type, duration.duration_minutes as parkland_time
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations` as proptype LEFT OUTER JOIN `bigquery-public-data.austin_bikeshare.bikeshare_trips` as duration
    ON proptype.station_id = duration.start_station_id
WHERE proptype.property_type = 'parkland')

,parkland_avg_min
0,40.986955


Here we can see that the trips that were on parkland had longer avg rides.